In [1]:
### import all items needed for this 

import requests
import os
import json
from twitter_api_keys import bearer_token
from pdb import set_trace
import pandas as pd
import ast
import yaml
import matplotlib.pyplot as plt
import csv
import numpy as np
import math
from scipy import stats
from collections import Counter

# sentiment analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
##analyzer = SentimentIntensityAnalyzer()

In [2]:
## helper function to get user ids

def get_twitter_user_ids(user):
    
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    
    url_user = "https://api.twitter.com/2/users/by/username/{}".format(user)
    
    response = requests.request("GET", url_user, headers=headers)
    a = response.json()
    return response.json()["data"]["id"]

In [3]:
### helper functions to add keys to tweet dictionaries

def add_key_entity_name(d, i):
    d["entity_name"] = i
    return d

def add_key_entity_type(d, i):
    d["entity_type"] = i
    return d

In [4]:
### create url of mentions

def create_twitter_url_mentions(mention):
    max_results = 100
    mrf = "max_results={}".format(max_results)
    q = "query=" + mention
    url = "https://api.twitter.com/2/tweets/search/recent?{}&{}%20lang%3Aen".format(
        mrf, q)
    url2 = "https://api.twitter.com/2/users/{}/mentions?max_results=100".format(mention)
    return url2


def twitter_auth_and_connect(bearer_token, url):
    ## query all parameters of a tweet

    query_params = {
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'pagination_token': {}}
    
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    
    
    for i in range(0, 10): ## each query returns up to 1000 tweets
        
        ## for the first response, do a full query of parameters
        if i == 0:
            response = requests.request("GET", url, headers=headers, params=query_params)
            m = response.json()
            print(response)
            d = m['data']

        if i > 0:
            
            try:
                
                ## for the next response, use pagination to iterate over results
                query_params["pagination_token"] = m['meta']["next_token"]
                print(m['meta']["next_token"])
                
                ### new request with the pagination
                response = requests.request("GET", url, headers=headers, params=query_params)
                print(response)
                
                ## add response to list of tweets
                d.extend(response.json()["data"])
                print(len(d))
                
                m = response.json()
                print("NEW PAGE") # print results to guide the process
                
            except:
                break
              
        
    return d

# twitter_auth_and_connect(bearer_token,"https://api.twitter.com/2/tweets/search/recent?max_results=100&query=Walmart")

In [5]:
### read the big and companies off of the excel sheet
### big companies are first 100, small companies are last 100 (out of 1000)

def get_companies():
    small = []
    big = []
    companies = pd.read_excel('entities.xlsx', sheet_name = [0])[0]

    for i in range(0, companies.shape[0]):
            if (i <= 99) & (companies['handle'][i] != 'na') & (str(companies['handle'][i]) !=  "nan"): #20 
                big.append(companies['handle'][i])
            if (i >= 899) & (companies['handle'][i] != 'na') & (str(companies['handle'][i]) !=  "nan"): #979
                small.append(companies['handle'][i])
                
#     print(big)
    print(len(big))
#     print(small)
    print(len(small))
    
    return big, small

big, small = get_companies()

99
84


In [6]:
## read other entities (such as performing arts)

companies = pd.read_excel('entities.xlsx', sheet_name = "Performing Arts")
handles = list(companies.handle)

In [80]:
### scraper block

all_tweets_list = [] ## list of tweets (raw)
ent_type = "performing_arts" ## the type of entity you are working on

user_id_dict = {}

### you start with the full list handles and then you continue, indexing on handles as needed (when the code breaks)
### reasons code may break: twitter handle was changed or deleted, twitter handle was banned
### I prefer for the code to break so it forces active monitoring of the code
###for i in handles:

### index on where the code broke and continue
for i in handles[49:]:
    print(i) ## print responses so you see what's happening. if response is not 200, code will break and then you troubleshoot
    a = get_twitter_user_ids(i)
    user_id_dict[i] = a
    url = create_twitter_url_mentions(a)
    tweets = twitter_auth_and_connect(bearer_token, url)
    tweets = [add_key_entity_name(c,i) for c in tweets]
    tweets = [add_key_entity_type(c,ent_type) for c in tweets]
    all_tweets_list.extend(tweets)

grotonhillmusic
<Response [200]>
7140dibdnow9c7btw3q4tl0cv013z2i01siy15yzvurea
<Response [200]>
197
NEW PAGE
stgpresents
<Response [200]>
7140dibdnow9c7btw4232j8qbvadmzuxzg8da6xr1q9mg
<Response [200]>
200
NEW PAGE
7140dibdnow9c7btw422noa4d2kg9munxhgwc8gmcamnq
<Response [200]>
300
NEW PAGE
7140dibdnow9c7btw42285xmaku88u5wpm5fmjos3xw6t
<Response [200]>
400
NEW PAGE
7140dibdnow9c7btw421t4kto6p6qoddktujwhz8ptd87
<Response [200]>
500
NEW PAGE
7140dibdnow9c7btw421e5d2eo792wzgibw99vdp1s2t0
<Response [200]>
600
NEW PAGE
7140dibdnow9c7btw420z41rlun57v98njst29pwnafco
<Response [200]>
700
NEW PAGE
7140dibdnow9c7btw420ytcxqzr50g36h7ce17zlq3ghl
<Response [200]>
800
NEW PAGE
7140dibdnow9c7btw420yr70kjnd29euspgo881rzb0a8
<Response [200]>
815
NEW PAGE
DallasSymphony
<Response [200]>
7140dibdnow9c7btw4232ni6ji6bzmkor15zw2pdddjcg
<Response [200]>
200
NEW PAGE
7140dibdnow9c7btw4228a5klku0nex9xgm8g7g91z6es
<Response [200]>
300
NEW PAGE
7140dibdnow9c7btw421ebrjn3gl6gyxq48ks5209h44w
<Response [200]>
400
NEW

<Response [200]>
300
NEW PAGE
7140dibdnow9c7btw421dwvcz3ke99b3qs8mhi4ppov93
<Response [200]>
400
NEW PAGE
7140dibdnow9c7btw420yr70m1uh0bzkb1x6iy5j7p0z4
<Response [200]>
500
NEW PAGE
7140dibdnow9c7btw420ju3ohwqqkkv82ss3mq3hdspiu
<Response [200]>
600
NEW PAGE
7140dibdnow9c7btw3z4k8cb6ln66crfbknpsfzt4o3zq
<Response [200]>
700
NEW PAGE
7140dibdnow9c7btw3z4593y04ottpk5poieqgcjvozli
<Response [200]>
800
NEW PAGE
7140dibdnow9c7btw3z3q9ws2dmruarfazn3g6sfxho39
<Response [200]>
831
NEW PAGE
DetroitSymphony
<Response [200]>
7140dibdnow9c7btw421taxi60uohrhqjnmofu08a1tti
<Response [200]>
200
NEW PAGE
7140dibdnow9c7btw420ytbg0hp6eipr5i3qisv6c89ps
<Response [200]>
300
NEW PAGE
7140dibdnow9c7btw420jjiqv3qn61tbw5xoj6zmav41d
<Response [200]>
400
NEW PAGE
7140dibdnow9c7btw3z3px3iol5n1bb4f243h5t2zdt84
<Response [200]>
500
NEW PAGE
7140dibdnow9c7btw3z2vo1v40wqeiqgc7b3r0d39ktv6
<Response [200]>
581
NEW PAGE
MTC_NYC
<Response [200]>
7140dibdnow9c7btw4232poegw3f2va2h1y9y05w8odh0
<Response [200]>
184
NEW PAGE


<Response [200]>
200
NEW PAGE
7140dibdnow9c7btw3z4k43gjuct76but5i2uxegbnqr9
<Response [200]>
300
NEW PAGE
7140dibdnow9c7btw3z3puyhqsyuthl8tpllxy8siw8c5
<Response [200]>
400
NEW PAGE
7140dibdnow9c7btw3z2gt15s952kifmwvbwy4ws3u0c7
<Response [200]>
500
NEW PAGE
7140dibdnow9c7btw3w4sktf6c8ppiz3jfp0i1xodbq5s
<Response [200]>
600
NEW PAGE
7140dibdnow9c7btw3w3ybp1zq5j5v51lyehi5zrb3k8g
<Response [200]>
700
NEW PAGE
7140dibdnow9c7btw3w33u5eepq1cl0b913nbu3xf2r00
<Response [200]>
800
NEW PAGE
7140dibdnow9c7btw3w1uhl0m1lrst0e5mlgrs9thbrco
<Response [200]>
846
NEW PAGE
fordstheatre
<Response [200]>
7140dibdnow9c7btw422nfs3vvm88t84ar5o7vfhg0som
<Response [200]>
200
NEW PAGE
7140dibdnow9c7btw4228efx80owrk91deoa5dr8mlhrq
<Response [200]>
300
NEW PAGE
7140dibdnow9c7btw421t8v6c4j471o58ixq3109t9quq
<Response [200]>
400
NEW PAGE
7140dibdnow9c7btw421t6pungnnncvnk2vrjbev4cyew
<Response [200]>
500
NEW PAGE
7140dibdnow9c7btw421t4lf4h5ksbhvr3qzd6xpqh9h4
<Response [200]>
600
NEW PAGE
7140dibdnow9c7btw421t2geiv82

<Response [200]>
400
NEW PAGE
7140dibdnow9c7btw421e5crkqug7on92qlm3dhaz0njm
<Response [200]>
500
NEW PAGE
7140dibdnow9c7btw420z89fgkvwf4ho7k8aizxemopbl
<Response [200]>
600
NEW PAGE
7140dibdnow9c7btw420yp2vvi58z0zqdirhvde2ryv5d
<Response [200]>
700
NEW PAGE
7140dibdnow9c7btw420jryy6hne3to49fjte5hviyqew
<Response [200]>
800
NEW PAGE
7140dibdnow9c7btw3z4keq6snbay7luqilpk07oukeut
<Response [200]>
821
NEW PAGE
MannCenter
<Response [200]>
7140dibdnow9c7btw4232rs8gjof7ph4um1jaoo8ibm0k
<Response [200]>
200
NEW PAGE
7140dibdnow9c7btw4232jattcpiry7z58i9s5zm9l3jo
<Response [200]>
300
NEW PAGE
7140dibdnow9c7btw4232h4w7pjdmongiwf7wxxurknqs
<Response [200]>
400
NEW PAGE
7140dibdnow9c7btw422nob0iqqar5zxu585dc464c2lx
<Response [200]>
500
NEW PAGE
7140dibdnow9c7btw422nftlx30o84kp11alo78dxqi46
<Response [200]>
600
NEW PAGE
7140dibdnow9c7btw422ndmha8acumwqdrlpjut0bb7zk
<Response [200]>
700
NEW PAGE
7140dibdnow9c7btw422nbi1xck6yhgf35i8fw3tnqnuh
<Response [200]>
800
NEW PAGE
7140dibdnow9c7btw4228p3jx50k7e

In [79]:
### here I changed a handle name because it was changed

handles.index("IndianHillMusic")
handles[49] = "grotonhillmusic"



In [81]:
### save the response of a list of dictionaries as jsons

with open("all_{}_tweets_1000_each_49_thru_end.json".format(ent_type), "w") as outfile:
    json_object = json.dumps(all_tweets_list, indent=4)
    outfile.write(json_object)